## RandomForest

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('C:/Users/ssj01/연구실 프로젝트/Data Set/전처리최종이었음좋겠다.csv')

X = data.drop(['한국표준산업분류코드10차(중분류)', '수익률 (1개월)(%)', '뉴스심리지수'], axis=1)
y = data['수익률 (1개월)(%)']
ym = data['yearmonth']
tk = data['Ticker']

In [ ]:
# cross_validate 적용
import warnings
warnings.filterwarnings("ignore")

from hyperopt import hp
from hyperopt import STATUS_OK
from hyperopt.early_stop import no_progress_loss

from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import joblib

a = '2004|2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020'
ya0 = []
tk0 = []
ym0 = []
ya1 = []

for i in tqdm(range(9)): 
    # tX = X[ym.str.contains(a[:24+5*i])]
    # vX = X[ym.str.contains(a[25+5*i:39+5*i])]
    tvX = X[ym.str.contains(a[:39+5*i])]
    tvym = ym[ym.str.contains(a[:39+5*i])]
    oX = X[ym.str.contains(a[40+5*i:44+5*i])]
    # ty = y[ym.str.contains(a[:24+5*i])]
    # vy = y[ym.str.contains(a[25+5*i:39+5*i])]
    tvy = y[ym.str.contains(a[:39+5*i])]
    oy = y[ym.str.contains(a[40+5*i:44+5*i])]
    otk = tk[ym.str.contains(a[40+5*i:44+5*i])]
    oym = ym[ym.str.contains(a[40+5*i:44+5*i])]
    
    # 하이퍼파라미터 검색 공간 설정
    rf_search_space = {
        # 노드를 분할하기 위한 최소한의 샘플 데이터 수 : 작게 설정할수록 분할되는 노드가 많아져서 과적합 가능성 증가 (default=2)
        'min_samples_split' : hp.quniform('min_samples_split', 2, 10, 2), # 8, 20, 4
        'max_depth' : hp.quniform('max_depth', 1, 6, 1)
    }
    
    # 목적 함수 정의
    def objective_func(search_space):
        rf = RandomForestRegressor(
            n_estimators = 500,
            min_samples_split = int(search_space['min_samples_split']),
            max_depth = int(search_space['max_depth']),
            random_state = 1 
        )
        
        r = 1
        
        # time series cv
        for j in range(3): # 마지막 3년만 valid로 사용
            tX = X[ym.str.contains(a[:24+5*(j+i)])]
            vX = X[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            ty = y[ym.str.contains(a[:24+5*(j+i)])]
            vy = y[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            
            # for문에 필요한 key를 미리 만듦
            ym_temp = vX['yearmonth'].reset_index(drop=True) # t = ym_temp==k를 위해 reset index
            key_yearmonth = dict.fromkeys(ym_temp)
            
            # vy의 index를 reset
            vy.reset_index(drop=True, inplace=True) # r = r*(vy[sort[-d:].index].mean()+100)/100를 위해 reset index
            
            # 년도, 티커 드랍
            tX = tX.drop(['yearmonth', 'Ticker'], axis=1)
            vX = vX.drop(['yearmonth', 'Ticker'], axis=1)
            
            # fitting
            rf.fit(tX, ty)
            
            # 수익률 metric 계산           
            y_pred = pd.Series(rf.predict(vX)) # sort_values() 사용을 위해 변환
    
            for k in list(key_yearmonth):
                t = ym_temp==k # 해당 month
                d = round(len(y_pred[t])/10)
                sort = y_pred[t].sort_values()
                r = r*(vy[sort[-d:].index].mean()+100)/100
                
        # 새로운 loss 정의
        new_loss = r*(-1)
        
        return {
            'loss':new_loss,
            'status':STATUS_OK
        }
    
    # fmin()을 사용하여 최적 하이퍼 파라미터 찾기
    from hyperopt import fmin, tpe, Trials
    import warnings
    warnings.filterwarnings('ignore')

    trial_val = Trials() # 결과 저장
    
    best = fmin(
        fn=objective_func,
        space=rf_search_space,
        algo=tpe.suggest,
        max_evals=1000,    # 최대 반복 횟수 지정
        trials=trial_val,
        early_stop_fn=no_progress_loss(20), # 30회 동안 개선되지 않으면 조기중지
        # https://github.com/hyperopt/hyperopt/issues/689
        rstate=np.random.default_rng(42) # https://github.com/hyperopt/hyperopt/issues/838
    )
    
    # 정수형 하이퍼 파라미터 값이 뒤에 .0이 붙어서 실수형으로 반환됨을 유의하기!
    best['min_samples_split'] = int(best['min_samples_split'])
    best['max_depth'] = int(best['max_depth'])
    print(best) 
    
    reg0 = RandomForestRegressor(
        n_estimators = 500,
        max_features = "sqrt",
        random_state = 1, 
        **best)
    
    import warnings
    warnings.filterwarnings('ignore')
    
    tvX = tvX.drop(['yearmonth', 'Ticker'], axis=1)
    oX = oX.drop(['yearmonth', 'Ticker'], axis=1)
    
    reg0.fit(tvX, tvy)
    
    yy0 = reg0.predict(oX)
    ya0 = ya0 + oy.tolist()
    ya1 = ya1 + yy0.tolist()
    tk0 = tk0 + otk.tolist()
    ym0 = ym0 + oym.tolist()
    joblib.dump(reg0,'RF_bayesian_last3_'+str(i)+'.pkl')
    
RF = pd.DataFrame({'real':ya0,'predict':ya1,'ticker':tk0,'yearmonth':ym0})
RF.to_csv('C:/Users/ssj01/연구실 프로젝트/Data Set/RF_bayesian.csv')    

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]


  2%|▊                                     | 20/1000 [4:34:01<223:47:13, 822.07s/trial, best loss: -2.9984755111479564]
{'max_depth': 4, 'min_samples_split': 8}


 11%|████████▍                                                                   | 1/9 [4:35:19<36:42:33, 16519.23s/it]


  4%|█▎                                  | 36/1000 [11:26:35<306:25:13, 1144.31s/trial, best loss: -2.6298791303056364]
{'max_depth': 6, 'min_samples_split': 10}


 22%|████████████████▋                                                          | 2/9 [16:04:00<60:29:20, 31108.61s/it]

...

 78%|█████████████████████████████████████████████████████████▌                | 7/9 [105:39:59<36:55:55, 66477.92s/it]


  4%|█▎                                   | 35/1000 [23:16:20<641:39:12, 2393.73s/trial, best loss: -2.012202328778012]
{'max_depth': 6, 'min_samples_split': 4}


 89%|█████████████████████████████████████████████████████████████████▊        | 8/9 [129:00:19<20:01:02, 72062.42s/it]


  3%|▉                                    | 27/1000 [19:43:24<710:46:18, 2629.78s/trial, best loss: -1.537729970822872]
{'max_depth': 6, 'min_samples_split': 8}


100%|█████████████████████████████████████████████████████████████████████████████| 9/9 [148:48:05<00:00, 59520.60s/it]


In [ ]:
import pandas as pd
import numpy as np

ya0 = pd.Series(RF['real'])
ya1 = pd.Series(RF['predict'])
tk0 = pd.Series(RF['ticker'])
ym0 = pd.Series(RF['yearmonth'])

r1 = []
r2 = []
for i in list(dict.fromkeys(ym0)):
    t = ym0==i
    d = round(len(ya1[t])/10)
    sort = ya1[t].sort_values()
    r1.append(ya0[sort[-d:].index].mean())
    r2.append(ya0[sort[0:d].index].mean())

rr1 = 1
rrf1 = [1]
for i in r1:
    rr1 = rr1*(i+100)/100
    rrf1.append(rr1)
rr2 = 1
rrf2 = [1]
for i in r2:
    rr2 = rr2*(i+100)/100
    rrf2.append(rr2)

In [ ]:
rrf1[-1]

30.867368644489456

In [ ]:
r1 = []
for i in list(dict.fromkeys(ym0)):
    t = ym0==i
    d = round(len(ya1[t])/10)
    sort = ya1[t].sort_values()
    r1.append(ya0[sort[-d:].index].mean())
    r2.append(ya0[sort[0:d].index].mean())

for i in range(9):
    rr1 = 1
    for j in r1[0+i*12:12+i*12]:
        rr1 = rr1*(j+100)/100
    print("year:" ,i, "test 수익률:", rr1)

year: 0 test 수익률: 1.5816503721627395
year: 1 test 수익률: 1.428531351070804
year: 2 test 수익률: 1.5675099395703078
year: 3 test 수익률: 1.6518959771565298
year: 4 test 수익률: 1.542127023777826
year: 5 test 수익률: 1.2655702560714341
year: 6 test 수익률: 1.2830993308755834
year: 7 test 수익률: 1.2275009248224207
year: 8 test 수익률: 1.7163973383917834
